# Generate *de novo* antibodies using Colab and RFantibody
This notebook walks you through the steps to generate novel antibodies using diffusion.

It uses the RFantibody model from [Rosetta Commons](https://github.com/RosettaCommons/RFantibody), described in detail in [this paper](https://www.biorxiv.org/content/10.1101/2024.03.14.585103v1).

Read the [accompanying blog](https://andrew-smith.me/blog?antibody).

Notebook by [Andrew Smith](https://www.linkedin.com/in/andrew-smith-8700a2219/).


In [ ]:
#@title 1. Verify your runtime has CUDA availability

!nvidia-smi > /dev/null 2>&1 && echo "✅ GPU Found" || echo "❌ Please connect to a GPU runtime to use this notebook."

## Environment Setup (Docker-Free RFantibody)

**What this cell does:**

Andrew Smith's workaround to run RFantibody without Docker on Colab:

1. **Poetry global install** — Disables virtualenv creation so all dependencies install into Colab's system Python (the Docker bypass)

2. **DGL wheel** — Precompiled Deep Graph Library for PyTorch 2.3 / CUDA 11.8 (graph neural network backend for RFdiffusion)

3. **cuda-python dependency dance** — Remove → install other deps → re-add. Workaround for Poetry dependency resolution conflicts.

4. **Model weights** — Downloads pretrained checkpoints for:
   - RFantibody (antibody-finetuned RFdiffusion)
   - ProteinMPNN (sequence design)
   - RoseTTAFold2 (structure validation)

5. **Example inputs:**
   - `6m0j_covid_spike.pdb` — SARS-CoV-2 RBD target antigen
   - `4nyl_HLT.pdb` — Adalimumab (Humira) framework in HLT format (Heavy/Light/Target chain annotation)

6. **Precompiled USalign** — Structure alignment binary (skips compile step)

**Potential gotcha:** DGL wheel pinned to CUDA 11.8. If CUDA mismatch errors occur, check Colab's CUDA version with `!nvcc --version`.

*Source: [Andrew Smith's blog](https://www.andrew-smith.me/blog/antibody/)*

In [ ]:
# Check available Python versions
!which python3.10
!python3.10 --version
!python3 --version

In [ ]:
#@title 2. Set Up Environment (Python 3.10 Fixed)
import os
import subprocess

print('Setting up Colab environment to run RFantibody...')

# Install basic tools in default Python (fine for these)
!pip install tqdm --quiet > /dev/null
!pip install wget --quiet > /dev/null
!pip install py3Dmol --quiet > /dev/null

# Bootstrap pip for Python 3.10
print("Bootstrapping pip for Python 3.10...")
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10

# — CONFIG —
REPO_DIR = "/content/RFantibody"
GIT_URL  = "https://github.com/RosettaCommons/RFantibody"

# Clone or update repo first
if not os.path.isdir(REPO_DIR):
    print("Cloning RFantibody repository...")
    subprocess.run(["git", "clone", GIT_URL], check=True)
else:
    print("Updating RFantibody repository...")
    subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

os.chdir(REPO_DIR)

# Download DGL wheel for Python 3.10
print("Downloading DGL wheel...")
!mkdir -p include/dgl
!wget -q https://data.dgl.ai/wheels/torch-2.3/cu118/dgl-2.4.0%2Bcu118-cp310-cp310-manylinux1_x86_64.whl \
    -O include/dgl/dgl-2.4.0+cu118-cp310-cp310-manylinux1_x86_64.whl

# Install all dependencies explicitly with python3.10 -m pip
print("Installing Python 3.10 dependencies (this takes a few minutes)...")
!python3.10 -m pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cu118 --quiet
!python3.10 -m pip install include/dgl/dgl-2.4.0+cu118-cp310-cp310-manylinux1_x86_64.whl --quiet
!python3.10 -m pip install hydra-core omegaconf scipy pandas biopython --quiet
!python3.10 -m pip install einops opt_einsum --quiet
!python3.10 -m pip install cuda-python --quiet

# Install local packages in editable mode
!python3.10 -m pip install -e /content/RFantibody/include/SE3Transformer --quiet
!python3.10 -m pip install -e /content/RFantibody --no-deps --quiet

# Download model weights
print("Downloading model weights...")
!bash include/download_weights.sh

# Get example files
import wget as wget_module
antibody_folder = "https://raw.githubusercontent.com/amerorchis/AntibodyFiles/refs/heads/main/"
files = ["6m0j_covid_spike.pdb", "4nyl_HLT.pdb"]

for f in files:
    out = f'/content/{f}'
    if not os.path.exists(out):
        try:
            wget_module.download(f'{antibody_folder}{f}', out=out)
        except Exception as e:
            print(f"Error downloading {f}: {e}")

# Get USalign binary
usalign_path = '/content/RFantibody/include/USalign/USalign'
if not os.path.exists(usalign_path):
    wget_module.download(f'{antibody_folder}USalign', out=usalign_path)
    os.chmod(usalign_path, 0o755)

print("\n✅ Setup complete!")

## Design Parameters

**Core inputs:**

| Parameter | Default | What it controls |
|-----------|---------|------------------|
| `target_pdb` | `6m0j_covid_spike.pdb` | Antigen structure — the thing you want to bind |
| `framework_pdb` | `4nyl_HLT.pdb` | Antibody scaffold (Adalimumab) with H/L/T chain annotations |
| `hotspot_res` | `E455,E456,E486,E489,E505` | Epitope residues — forces CDRs to contact these positions |
| `design_loops` | `L1:10-12,L2:7,L3:8-10,H1:7,H2:6,H3:12-16` | Allowed CDR loop lengths (min-max range per loop) |

**Diffusion parameters:**

| Parameter | Default | What it controls |
|-----------|---------|------------------|
| `num_designs` | 15 | Number of antibody backbones to generate |
| `T` | 100 | Total diffusion timesteps (more = finer sampling, slower) |
| `final_step` | 2 | When to stop denoising (lower = more refined) |
| `deterministic` | True | Seed RNG for reproducibility |

**Notes for scanner validation:**

- The `hotspot_res` values (E455, E456, E486, E489, E505) are ACE2-binding residues on the SARS-CoV-2 RBD — this targets the receptor binding site for potential neutralizing activity
- `design_loops` defines where ProteinMPNN will assign *de novo* sequences — **this is exactly where glycosylation liabilities can emerge**
- CDR-H3 (`H3:12-16`) has the widest length range and highest variability — historically the most common location for unexpected N-X-S/T sequons

In [ ]:
#@title 3. Set Parameters for Antibody Design {run: "auto"}

import ipywidgets as widgets
from IPython.display import display

# 1) Common style & layout
style  = {'description_width': '100px'}
layout = widgets.Layout(width='50%')

# 2) Define per-parameter descriptions (as HTML)
desc_target = widgets.HTML(
    "<b>Antigen PDB File Path</b><br>"
    "Full filesystem path to your antigen's PDB file. "
    "This will be used as the fixed target in the diffusion design."
)
desc_framework = widgets.HTML(
    "<b>Antibody Framework PDB File Path</b><br>"
    "Path to the antibody framework PDB (e.g. Fv, Fab). "
    "The new CDRs will be grafted onto this scaffold."
)
desc_hotspot = widgets.HTML(
    "<b>Epitope Residues</b><br>"
    "Comma-separated list of antigen residue IDs defining the hotspot "
    "region (e.g. E455,E456,…)."
)
desc_loops = widgets.HTML(
    "<b>CDR Loop Allowed Length</b><br>"
    "Specify allowed lengths per loop (L1-L3, H1-H3) in “min-max” format, "
    "comma-separated."
)
desc_num = widgets.HTML(
    "<b>Number of Designs to Generate</b><br>"
    "How many antibody designs the diffusion sampler should output."
)
desc_final = widgets.HTML(
    "<b>Final Diffusion Time Step</b><br>"
    "The last timestep index at which to apply the denoising network."
)
desc_T = widgets.HTML(
    "<b>Number of Diffusion Time Steps</b><br>"
    "Total timesteps in the forward noising chain (higher = finer control)."
)
desc_det = widgets.HTML(
    "<b>Make Runs Deterministic</b><br>"
    "If checked, seeds the RNG so you get repeatable results each run."
)

# 3) Initialize variables
target_pdb = '/content/6m0j_covid_spike.pdb'
framework_pdb = '/content/4nyl_HLT.pdb'
hotspot_res = 'E455,E456,E486,E489,E505'
design_loops = 'L1:10-12,L2:7,L3:8-10,H1:7,H2:6,H3:12-16'
num_designs = 15
final_step = 2
T = 100
deterministic = True

# 4) Create the widgets
target_pdb_w = widgets.Text(
    value=target_pdb,
    description='target_pdb:',
    style=style, layout=layout
)
framework_pdb_w = widgets.Text(
    value=framework_pdb,
    description='framework_pdb:',
    style=style, layout=layout
)
hotspot_res_w = widgets.Text(
    value=hotspot_res,
    description='hotspot_res:',
    style=style, layout=layout
)
design_loops_w = widgets.Text(
    value=design_loops,
    description='design_loops:',
    style=style, layout=layout
)
num_designs_w = widgets.IntSlider(
    value=num_designs, min=1, max=100, step=1,
    description='num_designs',
    style=style, layout=layout
)
final_step_w = widgets.IntSlider(
    value=final_step, min=1, max=50, step=1,
    description='final_step',
    style=style, layout=layout
)
T_w = widgets.IntSlider(
    value=T, min=1, max=200, step=1,
    description='T:',
    style=style, layout=layout
)
deterministic_w = widgets.Checkbox(
    value=deterministic,
    description='deterministic:',
    style=style, layout=layout
)

# 5) Build small VBox for each param + its description
boxes = [
    widgets.VBox([desc_target,    target_pdb_w]),
    widgets.VBox([desc_framework, framework_pdb_w]),
    widgets.VBox([desc_hotspot,   hotspot_res_w]),
    widgets.VBox([desc_loops,     design_loops_w]),
    widgets.VBox([desc_num,       num_designs_w]),
    widgets.VBox([desc_final,     final_step_w]),
    widgets.VBox([desc_T,         T_w]),
    widgets.VBox([desc_det,       deterministic_w]),
]

# 6) Callback that writes into values into global names
def _update(
    target_pdb,
    framework_pdb,
    hotspot_res,
    design_loops,
    num_designs,
    final_step,
    T,
    deterministic
):
    globals().update({
        'target_pdb':    target_pdb,
        'framework_pdb': framework_pdb,
        'hotspot_res':   hotspot_res,
        'design_loops':  design_loops,
        'num_designs':   num_designs,
        'final_step':    final_step,
        'T':             T,
        'deterministic': deterministic
    })

# 7) Wire interactive_output
controls = {
    'target_pdb':    target_pdb_w,
    'framework_pdb': framework_pdb_w,
    'hotspot_res':   hotspot_res_w,
    'design_loops':  design_loops_w,
    'num_designs':   num_designs_w,
    'final_step':    final_step_w,
    'T':             T_w,
    'deterministic': deterministic_w,
}
out = widgets.interactive_output(_update, controls)

# 8) Display the full form
display(widgets.VBox(boxes), out)


In [ ]:
!python3.10 -m pip install icecream --quiet

In [ ]:
!python3.10 -m pip install e3nn --quiet

In [ ]:
!python3.10 -m pip install pyrsistent --quiet

In [ ]:
#@title 4. Generate Antibodies with RFantibody
#@markdown ####(generation may take a while)
#@markdown ---
#@markdown #### Settings:
Verbose = True  #@param {type:"boolean"}

import shutil
import textwrap
from pathlib import Path
import os
from datetime import date

if os.getcwd() != '/content/RFantibody':
    os.chdir('/content/RFantibody')

# Move inference file to correct location.
shutil.copyfile("/content/RFantibody/scripts/rfdiffusion_inference.py", "/content/RFantibody/src/rfantibody/rfdiffusion/rfdiffusion_inference.py")
today = date.today().isoformat()

print("Generating antibody designs with RFdiffusion...")

# Set configuation for antibody generation run

PYTHONPATH         = '/content/RFantibody/include/SE3Transformer:/content/RFantibody/src:$PYTHONPATH'
pythonscript       = '/content/RFantibody/src/rfantibody/rfdiffusion/rfdiffusion_inference.py'
config_name        = 'antibody'
ckpt_override_path = '/content/RFantibody/weights/RFdiffusion_Ab.pt'
target_name        = target_pdb.split('/')[-1].split('.')[0]
output_folder      = f'outputs/{target_name}/{today}'
output_prefix      = f'{output_folder}/ab_des'

os.makedirs(output_folder, exist_ok=True)

# Interpolate the command with all settings
run_command = textwrap.dedent(f"""\
    export HYDRA_FULL_ERROR=1 && \
    PYTHONPATH={PYTHONPATH} \
    python3.10 {pythonscript} \
    --config-name {config_name} \
    antibody.target_pdb={target_pdb} \
    antibody.framework_pdb={framework_pdb} \
    inference.ckpt_override_path={ckpt_override_path} \
    ppi.hotspot_res=[{hotspot_res}] \
    antibody.design_loops=[{design_loops}] \
    inference.num_designs={num_designs} \
    inference.final_step={final_step} \
    diffuser.T={T} \
    inference.deterministic={deterministic} \
    inference.output_prefix={output_prefix} \
""").strip()

# Add output suppression if verbose is not selected
if not Verbose:
    run_command += " > /dev/null 2>&1"

# Execute the command
!{run_command}

print("\n✅ Run finished. Verifying output files...")

output_path = Path(output_prefix)
output_dir  = output_path.parent
prefix      = output_path.name

# grab all files like ab_des_*.pdb
generated = sorted(output_dir.glob(f"{prefix}_*.pdb"))

if len(generated) != num_designs:
    raise RuntimeError(
        f"❌ Expected {num_designs} designs, but found {len(generated)} files."
    )
else:
    print("✅ All outputs generated!")

In [ ]:
!ls -la /content/RFantibody/outputs/6m0j_covid_spike/2026-01-02/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/RFantibody_outputs
!cp -r /content/RFantibody/outputs/6m0j_covid_spike/2026-01-02/* /content/drive/MyDrive/RFantibody_outputs/

In [ ]:
#@title 5. Rank Antibody Candidates by Mean pLDDT {run: "auto"}
import numpy as np

# Compute mean pLDDT from B‑factor column (cols 61–66 in PDB format)
plddt_scores = {}
for pdb_path in generated:
    vals = []
    with open(pdb_path, 'r') as fh:
        for line in fh:
            if line.startswith(("ATOM", "HETATM")):
                try:
                    vals.append(float(line[60:66]))
                except ValueError:
                    pass
    plddt_scores[pdb_path.name] = np.mean(vals) if vals else float('-inf')

# Sort by descending pLDDT
ranked = sorted(plddt_scores.items(), key=lambda kv: kv[1], reverse=True)

class AntibodyDesign:
    def __init__(self, file, score, rank):
        self.file = f'{output_folder}/{file}'
        self.score = score
        self.rank = rank
        self.number = int(file.split('_')[2].replace('.pdb',''))

    def __str__(self):
        return f'{self.rank}. Design {self.number} ({self.score:.3f} mean pLDDT)'

antibodies = []
print("🏆 Antibody candidates ranked by mean pLDDT:")
for i, (name, score) in enumerate(ranked):
    print(f"{i+1}.  {name}: {score:.3f}")
    antibodies.append(AntibodyDesign(name, score, i+1))

In [ ]:
#@title 6. 3D Visualization of Generated Antibodies {run: "auto"}
import py3Dmol
from pathlib import Path
from ipywidgets import widgets
from IPython.display import display

# --- Dropdown to select which design to view ---
if not antibodies:
    print("❌ No design files found. Please ensure the generation step was successful.")
else:
    dropdown = widgets.Dropdown(
        options=[(str(p), p.file)
                 for p in antibodies],
        description='Design:',
    )

    # --- Legend widget ---
    legend = widgets.HTML(
        value="""
        <div style="display:flex; gap:1em; align-items:center; margin-top:8px;">
          <div style="width:12px; height:12px; background:steelblue;"></div>
          <span>Antibody (heavy chain)</span>
          <div style="width:12px; height:12px; background:forestgreen;"></div>
          <span>Antibody (light chain)</span>
          <div style="width:12px; height:12px; background:lightgrey; opacity:0.75;"></div>
          <span>Target (antigen)</span>
          <div style="width:12px; height:12px; background:red;"></div>
          <span>Epitope Hotspots</span>
        </div>
        """
    )

    # --- Visualization Function ---
    def visualize_pdb(pdb_file_path):
        """Creates an interactive 3D view of the antibody–antigen complex."""
        view = py3Dmol.view(width=800, height=600)
        pdb_content = Path(pdb_file_path).read_text()
        view.addModel(pdb_content, 'pdb')

        # Antibody as colored cartoons
        view.setStyle({'chain':['H']},
                      {'cartoon': {'color': 'steelblue'}})
        view.setStyle({'chain':['L']},
                {'cartoon': {'color': 'forestgreen'}})

        # Target surface
        view.addSurface(py3Dmol.VDW,
                        {'color': 'lightgrey', 'opacity': 0.75},
                        {'chain': 'T'})

        # Hotspots
        # (you’ll need to define hotspot_res somewhere above)

        hotspot_residues = [
            int("".join(filter(str.isdigit, r))) for r in hotspot_res.split(',')
        ]
        # Offset to account for renumbering
        if target_pdb == '/content/6m0j_covid_spike.pdb':
            hotspot_residues = [r - 97 for r in hotspot_residues]
        view.addStyle({'chain':'T','resi': hotspot_residues},
                      {'sphere': {'color': 'red', 'radius': 1.5}})

        view.zoomTo()
        view.show()

    # --- Display everything ---
    output = widgets.interactive_output(visualize_pdb,
                                        {'pdb_file_path': dropdown})
    display(dropdown, output, legend)

In [ ]:
!pip install Bio

In [ ]:
!pip install biopython --quiet


In [ ]:
# Extract sequences from the designed antibodies
from Bio.PDB import PDBParser
from Bio.SeqUtils import seq1

parser = PDBParser(QUIET=True)
output_dir = "/content/RFantibody/outputs/6m0j_covid_spike/2026-01-02/"

for i in range(15):
    pdb_file = f"{output_dir}ab_des_{i}.pdb"
    structure = parser.get_structure(f"ab_{i}", pdb_file)

    for model in structure:
        for chain in model:
            seq = ""
            for residue in chain:
                if residue.id[0] == " ":  # standard residue
                    seq += seq1(residue.resname)
            print(f"ab_des_{i} Chain {chain.id}: {seq[:80]}...")  # first 80 chars

In [ ]:
# Extract sequences from PDB files without biopython
import os

output_dir = "/content/RFantibody/outputs/6m0j_covid_spike/2026-01-02/"

three_to_one = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
}

for i in range(15):
    pdb_file = f"{output_dir}ab_des_{i}.pdb"
    chains = {}

    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith('ATOM') and line[12:16].strip() == 'CA':  # CA atoms only
                chain = line[21]
                resname = line[17:20].strip()
                resnum = int(line[22:26])

                if chain not in chains:
                    chains[chain] = {}
                if resnum not in chains[chain]:
                    chains[chain][resnum] = three_to_one.get(resname, 'X')

    print(f"\n=== ab_des_{i}.pdb ===")
    for chain_id in sorted(chains.keys()):
        seq = ''.join(chains[chain_id][r] for r in sorted(chains[chain_id].keys()))
        print(f"Chain {chain_id}: {seq}")

## Step 7: Sequence Design with ProteinMPNN

**What this does:**

RFdiffusion generated backbone geometries (3D coordinates) with poly-glycine placeholders in the CDR loops. ProteinMPNN now solves the *inverse folding problem*: given a desired backbone shape, what amino acid sequence will fold into that shape?

**The pipeline gap this exposes:**

ProteinMPNN optimizes for:
- ✅ Structural stability (will this sequence fold correctly?)
- ✅ Shape complementarity to target
- ❌ **NOT** glycosylation risk
- ❌ **NOT** developability liabilities
- ❌ **NOT** manufacturing considerations

This is exactly where N-X-S/T sequons can silently appear in CDR loops — ProteinMPNN has no awareness that it might be creating a glycosylation site that will wreck your titers in CHO cells.

**This is the AntibodyML value proposition:** Catch what ProteinMPNN doesn't know to avoid.

**Expected runtime:** 5-15 minutes for 15 designs (much faster than diffusion)

**Output:** PDB files with real amino acid sequences assigned to CDR loops, ready for scanner analysis

In [ ]:
#@title 7. Design Sequences with ProteinMPNN
import os

output_dir = "/content/RFantibody/outputs/6m0j_covid_spike/2026-01-02"
mpnn_output_dir = f"{output_dir}/mpnn_designs"
os.makedirs(mpnn_output_dir, exist_ok=True)

print("Running ProteinMPNN to assign sequences to CDR loops...")

!python3.10 /content/RFantibody/scripts/proteinmpnn_interface_design.py \
    -pdbdir {output_dir} \
    -outpdbdir {mpnn_output_dir}

print("\n✅ ProteinMPNN complete!")
!ls -la {mpnn_output_dir}

## find weights

In [ ]:
!find /content -name "*ProteinMPNN*.pt" 2>/dev/null
!find /content -name "*proteinmpnn*.pt" 2>/dev/null
!ls -la /content/RFantibody/weights/

In [ ]:
!mkdir -p /home/weights
!ln -sf /content/RFantibody/weights/ProteinMPNN_v48_noise_0.2.pt /home/weights/ProteinMPNN_v48_noise_0.2.pt

# Now re-run ProteinMPNN
print("Running ProteinMPNN to assign sequences to CDR loops...")

!python3.10 /content/RFantibody/scripts/proteinmpnn_interface_design.py \
    -pdbdir /content/RFantibody/outputs/6m0j_covid_spike/2026-01-02 \
    -outpdbdir /content/RFantibody/outputs/6m0j_covid_spike/2026-01-02/mpnn_designs

print("\n✅ ProteinMPNN complete!")
!ls -la /content/RFantibody/outputs/6m0j_covid_spike/2026-01-02/mpnn_designs

## Check for Possible Glycosyation

In [ ]:
#@title 8. Scan for N-linked Glycosylation Sequons (N-X-S/T, X≠P)
import re

sequences = {
    "ab_des_0": "EVQLVESGGGLVQPGRSLRLSCAASGTDLANGAMHWVRQAPGKGLEWVSAVDANGKGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARVSASTRSDIRGPLVGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKTSGAVGNTVGWYQQKPGKAPKLLIYNSSTRAGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYSLVTSGRHGFGQGTKVEIKRTV",
    "ab_des_1": "EVQLVESGGGLVQPGRSLRLSCAASGFDLATGAMHWVRQAPGKGLEWVSAIGSDGSGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCALGNRPTNSWRGNHPYGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITGKTSSASNHIGWYQQKPGKAPKLLIYDNSVRVGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYQMETAHRPKGFGQGTKVEIKRTV",
    "ab_des_2": "EVQLVESGGGLVQPGRSLRLSCAASGISLATGAMHWVRQAPGKGLEWVSAIDNDGSGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARAEDDGNLWGSSLSGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKTSYPNSRIGWYQQKPGKAPKLLISDVSIREGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYSQRNDSPRGFGQGTKVEIKRTV",
    "ab_des_3": "EVQLVESGGGLVQPGRSLRLSCAASGFDLSKGAMHWVRQAPGKGLEWVSAINASGSGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARQLYSRRSSKRYSALYGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKLSSSDSNIGWYQQKPGKAPKLLIYDTSQRAGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYQQNFSSTPMGFGQGTKVEIKRTV",
    "ab_des_4": "EVQLVESGGGLVQPGRSLRLSCAASGFDLSTGAMHWVRQAPGKGLEWVSAVDKSGGGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCALSTNGSLGTSALSGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKLSAPNPYVGWYQQKPGKAPKLLIYNISTRAGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYSWTSTLPYGFGQGTKVEIKRTV",
    "ab_des_5": "EVQLVESGGGLVQPGRSLRLSCAASGFDFTTGAMHWVRQAPGKGLEWVSAIDHDGKGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARSAGGDGDITDASLSGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKTSSATPAEKVGWYQQKPGKAPKLLIRSASERVGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYHSVGSDNGKLGFGQGTKVEIKRTV",
    "ab_des_6": "EVQLVESGGGLVQPGRSLRLSCAASGFNLSNGAMHWVRQAPGKGLEWVSATDSGGSGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCALSASGDISSPLSGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKTSSPVPSTSVGWYQQKPGKAPKLLISGASTRAGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYSQVSVNGRHGFGQGTKVEIKRTV",
    "ab_des_7": "EVQLVESGGGLVQPGRSLRLSCAASGFDLSAGAMHWVRQAPGKGLEWVSAIDADGTGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARGARNTSRHTTYNAPSGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKLSYPTPDYVGWYQQKPGKAPKLLIYNTSTRVGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYSMDYSSSPKGFGQGTKVEIKRTV",
    "ab_des_8": "EVQLVESGGGLVQPGRSLRLSCAASGLDLSAGAMHWVRQAPGKGLEWVSAIDANGKGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCALSADGAIGSELYGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITGKTSAPSSRIGWYQQKPGKAPKLLIYATSERAGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYQADFGNRPRGFGQGTKVEIKRTV",
    "ab_des_9": "EVQLVESGGGLVQPGRSLRLSCAASGFDFSKGAMHWVRQAPGKGLEWVSAIDADGKGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARAEGPSILSPLSGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKTSSPSDGAIGWYQQKPGKAPKLLIYRTSIRAGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYSADYSYNPKGFGQGTKVEIKRTV",
    "ab_des_10": "EVQLVESGGGLVQPGRSLRLSCAASGVDFSKGAMHWVRQAPGKGLEWVSAIDADGAGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCALSMFGSIDLADLYGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKTSYGVPASKVGWYQQKPGKAPKLLIYATSIRVGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYSAVGDYGKLGFGQGTKVEIKRTV",
    "ab_des_11": "EVQLVESGGGLVQPGRSLRLSCAASGINLNMGAMHWVRQAPGKGLEWVSAIDPDGDGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARDASSVGTLIPGGALAGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKTSSSTPNEVGWYQQKPGKAPKLLIYNSSTRAGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYSRTYRAPPAGFGQGTKVEIKRTV",
    "ab_des_12": "EVQLVESGGGLVQPGRSLRLSCAANDNDFPNGAMHWVRQAPGKGLEWVSAVWSNGVGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARGGGSGVGATSVLTGGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSSGSPVGSEAAGWYQQKPGKAPKLLIRGTVDLSGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYAYRGDVGDRSGFGQGTKVEIKRTV",
    "ab_des_13": "EVQLVESGGGLVQPGRSLRLSCAASGFDFRLGAMHWVRQAPGKGLEWVSAVDGNGVGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARSLTNYIDSAALSGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITGTTSSDTPNNVGWYQQKPGKAPKLLIYGTSTRAGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYQAAGRVGKGVGFGQGTKVEIKRTV",
    "ab_des_14": "EVQLVESGGGLVQPGRSLRLSCAASGVDFSKGAMHWVRQAPGKGLEWVSAISADGKGIDYADSVEGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCALSETGHLGSALVGWGQGTLVTVSSASDIQMTQSPSSLSASVGDRVTITSKTSYGSDLIGWYQQKPGKAPKLLIYRNSKRAGGVPSRFSGSGSGTDFTLTISSLQPEDVATYYSARFDTTPMGFGQGTKVEIKRTV",
}

def find_nxst_motifs(seq, name):
    """Find N-X-S/T motifs where X != P"""
    motifs = []
    for i in range(len(seq) - 2):
        if seq[i] == 'N' and seq[i+1] != 'P' and seq[i+2] in ['S', 'T']:
            motif = seq[i:i+3]
            # Get context (5 residues on each side)
            start = max(0, i-5)
            end = min(len(seq), i+8)
            context = seq[start:i] + f"[{motif}]" + seq[i+3:end]
            motifs.append((i+1, motif, context))  # 1-indexed position
    return motifs

print("=" * 70)
print("🔬 GLYCOSYLATION SEQUON SCAN (N-X-S/T, X≠P)")
print("=" * 70)

total_motifs = 0
designs_with_motifs = 0

for name, seq in sequences.items():
    motifs = find_nxst_motifs(seq.upper(), name)
    if motifs:
        designs_with_motifs += 1
        print(f"\n⚠️  {name}: {len(motifs)} potential N-glycosylation site(s)")
        for pos, motif, context in motifs:
            total_motifs += 1
            # Flag if in CDR region (rough estimate based on position)
            location = "Framework" if pos < 25 or (pos > 115 and pos < 140) else "CDR/Variable"
            print(f"   Position {pos}: {motif} | ...{context}... | {location}")
    else:
        print(f"\n✅ {name}: No N-X-S/T motifs found")

print("\n" + "=" * 70)
print(f"SUMMARY: {total_motifs} total motifs across {designs_with_motifs}/15 designs")
print("=" * 70)

## Results: Glycosylation Sequon Scan

### Basic N-X-S/T Motif Detection (Regex Only)

| Metric | Value |
|--------|-------|
| Designs scanned | 15 |
| Designs with N-X-S/T motifs | 7 (47%) |
| Total sequons detected | 10 |
| Location | 100% in CDR/Variable regions |

### Key Findings

ProteinMPNN introduced potential N-linked glycosylation sites in nearly half of all designs — with **zero awareness** that it was creating manufacturing liabilities.

**Hotspot patterns observed:**
- CDR-L3 region (~position 173-177): Recurrent `KLLIY[NXS]` motif
- CDR-H3 (~position 101-102): High variability = high risk
- CDR-H1/H2: Sporadic hits

### ⚠️ CRITICAL CAVEAT: This is NOT the full AntibodyML scan

This analysis used only a basic regex for canonical N-X-S/T (X≠P) motifs.

**The Enhanced Progenitor Glycosylation Scanner additionally detects:**
- 🧬 **Cryptic sequons** — one mutation away from N-X-S/T
- 🔄 **Progenitor motifs** — N-X-A, N-X-V that evolve into glycosylation sites during affinity maturation
- 🔬 **Structural accessibility** — is the site actually exposed to glycosyltransferases?
- 📐 **Vernier context effects** — framework residues that modulate CDR loop exposure

**The 47% detection rate is the floor, not the ceiling.**

Published validation shows current ML tools miss 79-86% of glycosylation risks. The full scanner would likely flag additional liabilities in the "clean" designs above.

### Bottom Line

> RFantibody + ProteinMPNN: State-of-the-art AI antibody design, published in *Nature* (Nov 2025)
>
> Result: 47% of designs have obvious glycosylation liabilities, 100% in the designed CDR loops
>
> **This is the gap AntibodyML fills.**